[Inhaltsverzeichnis](../AP4.ipynb) | [ < zurück ](wohin?)|  [ > next](wohin?)

## 4.3.1  Szenarien stündlich aufgelöst




FLUCCOplus Setup

In [7]:
# OPTIONAL: Load the "autoreload" extension so that code can change
import numpy as np
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [9]:
%matplotlib inline
from FLUCCOplus.notebooks import *



FLUCCOplus.notebooks loaded!


### Scenario Überblick
Zuerst werden alle Szenarien aus Excel eingelesen und dargestellt

In [10]:
import FLUCCOplus.scenarios as scens

In [11]:
sc = scens.all()
print(*[i+k for i,k in zip([i for i in sc.index], ["\n"]*len(sc.index))])

EM2018
 EM2019
 E-Control 2019
 Energie und Klimazukunft 2030 (Veigl17)
 Erneuerbare Energie 2030 (UBA16)
 WEM 2030 (UBA17)
 Transition 2030 (UBA17)
 Energie und Klimazukunft 2050 (Veigl17)
 Erneuerbare Energie 2050 (UBA16)
 WEM 2050 (UBA17)
 Transition 2050 (UBA17)
 100% Erneuerbare Deckung 2050 (FLUCCO+)
 100% Erneuerbare Deckung 2050 inkl Methan (FLUCCO+)
 100% Erneuerbare Deckung 2050 ohne Speicherausbau (FLUCCO+)



C:\Users\Simon Schneider\anaconda\envs\FLUCCOplus\lib\site-packages\openpyxl\worksheet\_reader.py:308: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Simon Schneider\anaconda\envs\FLUCCOplus\lib\site-packages\openpyxl\worksheet\_reader.py:308: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


## Creating a Scenario object



In [12]:
from FLUCCOplus.scenarios import Scenario
 

### EM Base data
Select an electricity map year


In [13]:
em_year = 2018 # 2015, 2016, 2017, 2018

Select an available Scenario ``name`` 

In [14]:
#name = "EM2018"
#name = "EM2019"                                                  
#name = "E-Control 2019"                                          
#name = "Energie und Klimazukunft 2030 (Veigl17)"                 
#name = "Erneuerbare Energie 2030 (UBA16)"                        
#name = "WEM 2030 (UBA17)"                                        
#name = "Transition 2030 (UBA17)"                                 
#name = "Energie und Klimazukunft 2050 (Veigl17)"                
#name = "Erneuerbare Energie 2050 (UBA16)"                        
#name = "WEM 2050 (UBA17)"                                       
name = "Transition 2050 (UBA17)"                                 
#name = "100% Erneuerbare Deckung 2050 (FLUCCO+)"                 
#name = "100% Erneuerbare Deckung 2050 inkl Methan (FLUCCO+)" 

Create the scenario

In [15]:
import FLUCCOplus.electricitymap as elmap

s = Scenario(  
    name = name, # can be anything
    scenario = name, # must be a scenario name
    em_base = {"year": em_year,
               "df": elmap.fetch(year=em_year)})
s
#s.plot()


C:\Users\Simon Schneider\anaconda\envs\FLUCCOplus\lib\site-packages\openpyxl\worksheet\_reader.py:308: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Simon Schneider\anaconda\envs\FLUCCOplus\lib\site-packages\openpyxl\worksheet\_reader.py:308: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Transition 2050 (UBA17)

Szenario                target          base      scale
Jahr               2050.000000   2018.000000        NaN
Strombedarf      115805.564820  63497.589935   1.823779
Mismatch              0.000000           NaN        NaN
Importe               0.000000           NaN        NaN
Stromproduktion  115805.564820           NaN        NaN
Wasserkraft       47005.003760           NaN        NaN
Pumpspeicher      15041.601203   4101.638049   3.667218
Laufkraft         31963.402557  30013.912729   1.064953
Windkraft         28350.002268   6396.599663   4.432043
Photovoltaik      26456.668783   1420.744677  18.621691
Volatile EE       86770.073608           NaN        NaN
Nicht-Volatile    29035.491212           NaN        NaN
RES0              29035.491212           NaN        NaN
RES1              13993.890008           NaN        NaN
RES2             -15041.601203           NaN        NaN

In [16]:
s.TSD["Strombedarf"].sum()

115805564.82000001

In [17]:
s.annuals

Szenario,target,base,scale
Jahr,2050.000000,2018.000000,NaN
Strombedarf,115805.564820,63497.589935,1.823779
Mismatch,0.000000,NaN,NaN
Importe,0.000000,NaN,NaN
Stromproduktion,115805.564820,NaN,NaN
Wasserkraft,47005.003760,NaN,NaN
Pumpspeicher,15041.601203,4101.638049,3.667218
Laufkraft,31963.402557,30013.912729,1.064953
Windkraft,28350.002268,6396.599663,4.432043
Photovoltaik,26456.668783,1420.744677,18.621691


<AxesSubplot:xlabel='datetime'>

In [18]:
s.base_df["power_production_solar_avg"].plot()

In [19]:
from FLUCCOplus.electricitymap import *

df_em = np.genfromtxt("../data/raw/electricityMap/Electricity_map_2015-2019.csv", delimiter=";")

## Stunde-Skalierung

Über acht 3-stündliche Werte wird ein spezifische Tagesverlauf generiert und für das ganze Jahr ...

In [20]:
# apply transform
import FLUCCOplus.transform as traffo
import FLUCCOplus.plots as plots
import os

if not os.path.exists("../data/interim"):
        os.makedirs("../data/interim")

hscale = 24
p_day = np.array([1.0, 1.1, 0.9, 0.9, 1.1, 1.0])
day = traffo.transform(p_day, hour_scale=hscale)

x = np.arange(0, hscale, hscale/len(p_day))
xh = np.arange(0,8760,1)


In [21]:
example = traffo.zyklusscaler(day, [10, 11, 10, 14, 13, 8, 10, 8, 12, 11, 10, 9], 24)  # funktioniert nur mit ganze Zahlen... :S
asdasd = traffo.transform(p_day, hour_scale=8760)
result = np.multiply(example, asdasd)
pd.DataFrame(example).plot()

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
plots.plot_analyse(df=example, df2=result, figsize=(14,7), start=0, stop=8760)

In [ ]:
day_to_30TWh = traffo.transform(p_day, hour_scale=8760)
pd.DataFrame(day).plot()

In [ ]:
df_scaled = s.TSD["Strombedarf"] * day
df_scaled.plot()
df_scaled.sum()

In [ ]:

x = np.arange(0, hscale, hscale/len(p_day_to_target))
xh = np.arange(0,8760,1)

plt.plot(xh[:hscale*2], day_to_target[:hscale*2], "purple")
plt.plot(x[:hscale*2], p_day_to_target[:hscale*2], "go")

In [ ]:
x = np.arange(0, hscale, hscale/len(p_day_to_target))
xh = np.arange(0,8760,1)

plt.plot(xh[:hscale*2], day[:hscale*2], "r")
plt.plot(xh[:hscale*2], day_to_target[:hscale*2], "purple")
plt.legend(['Defaultwert', 'Targetwert'])
#plt.plot(x[:hscale*2], p_day[:hscale*2], "bo")
#plt.plot(x[:hscale*2], p_day_to_target[:hscale*2], "go")
plt.savefig('../Data/interim/Default_and_Targetvalue.png', dpi=300)

Der skalierte Jahresverlauf wird mit dem Skalierverlauf multipliziert...

In [ ]:
scaled_day_to_target = np.multiply(day_to_target,day_scaler)
scaled_day_to_target

In [ ]:
scaled_day_to_target.sum()

In [ ]:
target_value

In [ ]:
x = np.arange(0, hscale, hscale/len(p_day_to_target))
xh = np.arange(0,8760,1)

plt.plot(xh[:hscale*2], day_to_target[:hscale*2], "r")
plt.plot(xh[:hscale*2], scaled_day_to_target[:hscale*2], "orange", linewidth=2.0)

plt.legend(['Zielwert', 'skalierte Zielwert'])
plt.plot(x[:hscale*2], p_day_to_target[:hscale*2], "bo")
plt.savefig('../Data/interim/skalierte_Default_and_Targetvalue.png', dpi=300)

In [ ]:
import panel as pn
import panel.widgets as pnw
from matplotlib.figure import Figure

def scaler_plot(scaler, support_points, hour_scale, view_scale=2):
    """
    scaler: zeitreihe (8760)
    
    """
    p, h = support_points, hour_scale

    x = np.arange(0, h, h/len(p))
    xh = np.arange(0,8760,1)

    fig = Figure()
    ax = fig.add_subplot()
    a, b = 0, int(h*view_scale)
    ax.plot(x[a:b], p[a:b], "bo")
    ax.plot(xh[a:b], scaler[a:b], "r")
    return fig

scaler_plot(day,p_day, 24)

In [ ]:
scaled_day

In [ ]:
day

In [ ]:
example = traffo.zyklusscaler(day, [10, 10, 14, 13, 8, 7, 9, 10], 12) # funktioniert nur mit ganze Zahlen
example

In [ ]:
x = np.arange(0, hscale, hscale/len(p_day_to_target))
xh = np.arange(0,8760,1)
pd.DataFrame(example)
x = np.arange(0, hscale, hscale/len(p_day_to_target))
xh = np.arange(0,8760,1)
plt.plot(xh[:hscale*2], day[:hscale*2], "b")
plt.plot(xh[:hscale*2], example[:hscale*2], "r")

In [ ]:
z=8760
timeseries = np.arange(0,8760, 1)
timeseries_grouped = np.arange(0,8760, 1)
fact = [1,2,3,4,5,6,7,8,9,10,11,12]
fact_up = np.arange(0,8760, 1)
g = int(len(timeseries)/z)
for i in range(len(fact)):
    fact_up[(i * int(z/len(fact))):((i + 1) * int(z/len(fact)))] = fact[i]
for j in range(g-1):
    fact_up[(j+1)*z:((j+2)*z)] = fact_up[0:z]

#for i in range(int(round(len(timeseries)/z))):
 #    timeseries_grouped= timeseries[(i*z):((i+1)*z)]*fact[i]
  #   timeseries_grouped2= timeseries[(i*z):((i+1)*z)]*fact[i-1]
for i in range(len(fact)):
    for j in range(z):
    #timeseries_grouped[i] = timeseries[(i * z):((i + 1) * z)]
    #timeseries_grouped = np.multiply(timeseries[(i*z):((i+1)*z)],fact[i])
        timeseries_grouped[j] = timeseries[j]* fact[i in range(z)]
#timeseries_grouped[i] = timeseries[(i*z):((i+1)*z)]*fact[i]
fact_up
#timeseries_grouped
#timeseries

In [ ]:
for i in range(len(fact)):
    #for j in range(int(round(len(timeseries)/z))):
        fact_up[(i * int(z/len(fact))):((i + 1) * int(z/len(fact)))] = fact[i]
fact_up

In [ ]:
#params
N = 12 # muss eigentlich anzahl der skalierungswerte sein
zyklus = 24 #h zyklus
timeseries = df_scaled


points = [pnw.FloatSlider(name='', value=1.0, start=0., end=3., step=0.05) for i in range(N)]
#points = np.array(p_day_scaler)
#points = [pnw.FloatSlider(name='', value=1.0, start=0., end=3., step=0.05) for i in range(N)]
@pn.depends(*points)
def reactive_scalar(*points):
    p = np.array(p_day_scaler)
    transformed_timeseries = timeseries * traffo.transform(p)
    return scaler_plot(transformed_timeseries, points, zyklus)

widgets   = pn.Column("<br>\n# Reactive Scaler")
sliders = pn.Column(widgets, *points)
dashboard = pn.Row(sliders, reactive_scalar)

pn.extension()
pn.interact()
dashboard

## Monate-Skalierung

In [ ]:
# scale to fit targetsum

# get load
# apply transform
#     ecars (non dsm)
#
# scale to fit targetsum

# em scenario residual load report